### Import Libraries

In [59]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from scipy import stats
import numpy as np

In [60]:
# Load the CSV file
data = pd.read_csv("philip.csv")
# check data type
data.dtypes

ApplicationDate                object
Age                             int64
AnnualIncome                    int64
CreditScore                     int64
EmploymentStatus               object
EducationLevel                 object
Experience                      int64
LoanAmount                      int64
LoanDuration                    int64
MaritalStatus                  object
NumberOfDependents              int64
HomeOwnershipStatus            object
MonthlyDebtPayments             int64
CreditCardUtilizationRate     float64
NumberOfOpenCreditLines         int64
NumberOfCreditInquiries         int64
DebtToIncomeRatio             float64
BankruptcyHistory               int64
LoanPurpose                    object
PreviousLoanDefaults            int64
PaymentHistory                  int64
LengthOfCreditHistory           int64
SavingsAccountBalance           int64
CheckingAccountBalance          int64
TotalAssets                     int64
TotalLiabilities                int64
MonthlyIncom

In [61]:
# Remove rows with missing values
data.dropna(inplace=True)
# Remove duplicate rows
data.drop_duplicates(inplace=True)
# Strip extra whitespace from string columns
for column in data.select_dtypes(include=['object']).columns:
    data[column] = data[column].str.strip()

In [62]:
# Check all the text data in Columns that we will apply Ordinal Encoder
ordinal_cols = ["HomeOwnershipStatus", "EmploymentStatus", "EducationLevel", "MaritalStatus"]
# Get unique values for each specified column
for column in ordinal_cols:
    unique_values = data[column].unique()
    print(f"Unique values in '{column}': {unique_values}")

Unique values in 'HomeOwnershipStatus': ['Own' 'Mortgage' 'Rent' 'Other']
Unique values in 'EmploymentStatus': ['Employed' 'Self-Employed' 'Unemployed']
Unique values in 'EducationLevel': ['Master' 'Associate' 'Bachelor' 'High School' 'Doctorate']
Unique values in 'MaritalStatus': ['Married' 'Single' 'Divorced' 'Widowed']


Based on general societal and financial perceptions, theses are the ranking for each category:
* HomeOwnershipStatus: Own > Mortgage > Rent > Other
* EmploymentStatus: Employed > Self-Employed > Unemployed
* EducationLevel: Doctorate > Master > Bachelor > Associate > High School
* MaritalStatus: Married > Widowed > Divorced > Single

In [68]:
# Columns to keep
columns_to_keep = [
    "BankruptcyHistory", "HomeOwnershipStatus", 
    "EmploymentStatus", "EducationLevel", "MaritalStatus",
    "LoanAmount", "NumberOfDependents", "MonthlyDebtPayments", 
    "DebtToIncomeRatio", "NumberOfCreditInquiries", "LengthOfCreditHistory", 
    "CheckingAccountBalance"
]

# Create an ordinal ranking for each column
ordinal_rankings = {
    "HomeOwnershipStatus": ["Own", "Mortgage", "Rent", "Other"],
    "EmploymentStatus": ["Employed", "Self-Employed", "Unemployed"],
    "EducationLevel": ["Doctorate", "Master", "Bachelor", "Associate", "High School"],
    "MaritalStatus": ["Married", "Widowed", "Divorced", "Single"]
}

# Check for NaN values in the ordinal columns
for col in ordinal_rankings.keys():
    if data[col].isnull().any():
        print(f"Warning: NaN values found in column '{col}'. Filling NaNs with 'Unknown'.")
        data[col].fillna('Unknown', inplace=True)  # Replace NaNs with a placeholder

# Ensure that the categorical columns are of type 'category'
for col in ordinal_rankings.keys():
    data[col] = data[col].astype('category')

# Verify the data types before encoding
print("Data types before encoding:")
print(data[ordinal_rankings.keys()].dtypes)

# Ordinal Encoding on specified columns
ordinal_cols = list(ordinal_rankings.keys())
ordinal_encoder = OrdinalEncoder(categories=list(ordinal_rankings.values()))

try:
    # Fit and transform the data, ensuring the output is an integer type
    encoded_values = ordinal_encoder.fit_transform(data[ordinal_cols])
    data[ordinal_cols] = encoded_values.astype(np.int64)  # Ensure the output is integer
except ValueError as e:
    print(f"Error during encoding: {e}")
    print("Check the data in the following columns:")
    for col in ordinal_cols:
        print(f"{col}: {data[col].unique()}")  # Print unique values in each column

# Select relevant columns and create a copy to avoid SettingWithCopyWarning
cleaned_data = data[columns_to_keep].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Step 1: Scaling the numerical columns
scaler = StandardScaler()
numeric_cols = ["LoanAmount", "NumberOfDependents", "MonthlyDebtPayments", 
                "DebtToIncomeRatio", "NumberOfCreditInquiries", "LengthOfCreditHistory", 
                "CheckingAccountBalance"]

cleaned_data.loc[:, numeric_cols] = scaler.fit_transform(cleaned_data[numeric_cols])  # Use .loc to avoid warning

# Step 2: Removing outliers (using z-score threshold of 3)
cleaned_data = cleaned_data[(np.abs(stats.zscore(cleaned_data[numeric_cols])) < 3).all(axis=1)]

# Step 3: Basic Analysis - Print summary statistics
print("Summary Statistics for Cleaned Data:")
print(cleaned_data.describe())

# Save the processed data to a new CSV
cleaned_data.to_csv("philip_fully_processed.csv", index=False)
print("Data fully processed and saved to 'philip_fully_processed.csv'")

Data types before encoding:
HomeOwnershipStatus    category
EmploymentStatus       category
EducationLevel         category
MaritalStatus          category
dtype: object
Error during encoding: invalid literal for int() with base 10: 'Own'
Check the data in the following columns:
HomeOwnershipStatus: [0, 1, 2, 3]
Categories (4, int64): [0, 1, 2, 3]
EmploymentStatus: [0, 1, 2]
Categories (3, int64): [0, 1, 2]
EducationLevel: [1, 3, 2, 4, 0]
Categories (5, int64): [0, 1, 2, 3, 4]
MaritalStatus: [0, 3, 2, 1]
Categories (4, int64): [0, 1, 2, 3]
Summary Statistics for Cleaned Data:
       BankruptcyHistory    LoanAmount  NumberOfDependents  \
count       18624.000000  18624.000000        18624.000000   
mean            0.053157     -0.063502            0.000845   
std             0.224353      0.856299            0.998881   
min             0.000000     -1.579559           -1.094504   
25%             0.000000     -0.700608           -1.094504   
50%             0.000000     -0.235931       